In [1]:
from scipy.sparse import csr_matrix, tril
from scipy.special import comb
import pandas as pd
import numpy as np
import math
import os
import json


In [2]:
### NEW RECOMMENDATION FUNCTION ###
def evaluate_recommendations(recommendations_df, masked_songs_df, list_of_ks = [10,20,30,40,50]):
    """
    Evaluate the recommendations using Recall@k.
    Each user should prefable have 50 recommendations.
    
    Parameters:
    - recommendations_df (pd.DataFrame): DataFrame with 'user_id' and 'recommended_songs' (list of song_ids).
    - masked_songs_df (pd.DataFrame): DataFrame with 'user_id' and 'song_id' of masked songs.
    - ks (list): List of integers with the values of k to compute Recall@k.
    
    Returns:
    - evaluation_df (pd.DataFrame): DataFrame with 'user_id', and 'k_i' columns for each k in ks.   
    """
    # Ensure the recommended_songs are lists
    recommendations_df['recommended_songs'] = recommendations_df['recommended_songs'].apply(list)
    
    # Group masked songs by user
    masked_songs_grouped = masked_songs_df.groupby('user_id')['song_id'].apply(set).reset_index()
    masked_songs_dict = dict(zip(masked_songs_grouped['user_id'], masked_songs_grouped['song_id']))
    
    evaluation_results = []
    
    for _, row in recommendations_df.iterrows():
        user_id = row['user_id']
        # Get the masked songs for the user
        masked_songs = masked_songs_dict.get(user_id, set())
        recalls = []
        for k in list_of_ks:
            recommended_songs = row['recommended_songs'][:k]   
            if not masked_songs:
                # If there are no masked songs for the user, we cannot compute recall
                recall_at_k = None
            else:
                # Compute the number of relevant recommended songs
                relevant_recommendations = set(recommended_songs) & masked_songs
                num_relevant = len(relevant_recommendations)

                # Recall@k
                recall_at_k = num_relevant / len(masked_songs) if len(masked_songs) > 0 else 0

            recalls.append(recall_at_k)

        recall_dict = {f"k_{k}": recall_at_k for k, recall_at_k in zip(list_of_ks, recalls)}

        evaluation_results.append({
            'user_id': user_id,
            **recall_dict
        })

    evaluation_df = pd.DataFrame(evaluation_results)
    return evaluation_df

# 6. Apriori Algorithm

In this section, we’ll use the Apriori Algorithm to identify frequent itemsets of size 2.
Based on the frequent itemsets, we will generate association rules to recommend songs to users based on the songs they have listened to. The association rules used are confidence and lift.


## 6.1 Sparse Matrix Representation

We use the `csr_matrix` class from the `scipy.sparse` module to represent the transaction matrix.
This allows us to save memory by only storing the non-zero elements of the matrix. The `csr_matrix` class is a sparse matrix representation that stores the matrix in Compressed Sparse Row format and it is efficient for matrix-vector multiplication, which is how we will find the frequent itemsets of size 2.

In [3]:
PATH_ALL_USERS_DATA = "data/processed/user_data_cleaned.csv"
PATH_TRAIN_USERS_IDS = "data/processed/train_users.csv"
PATH_TRAIN_USERS_DATA = "data/processed/user_data_clean_train.csv" # this will be created by the script

In [4]:
def find_train_users(path_to_all_users_data: str, path_to_train_users_IDs: str, output_path: str) -> pd.DataFrame:
    
    # Read the data files
    listened_songs_all_users = pd.read_csv(path_to_all_users_data)
    train_users = pd.read_csv(path_to_train_users_IDs)["user_id"]
    # Filter the dataframe to include only users in the training set
    train_users_data = listened_songs_all_users[listened_songs_all_users["user_id"].isin(train_users)]
    # Save the filtered dataframe    
    train_users_data.to_csv(output_path, index=False)
    # Print the number of unique users
    print(f"Number of unique users in the training set: {train_users_data['user_id'].nunique()}")
    print(f"Number of unique songs in the training set: {train_users_data['song_id'].nunique()}")
    return train_users_data   

def create_sparse_matrix(path_to_train_users_data: str, transaction_matrix: bool = True):
    """
    Create a sparse matrix from a csv file with user data with columns song_id, user_id, play_count    
    Input:
    path_to_user_data: str, path to the user data
    transaction_matrix: bool, if True the play_count is set to 1, else the play_count is used as is
    Returns:
    sparse_matrix: csr_matrix, 
    user_id_mapping: dict, mapping from the integer codes to the original user ids
    song_id_mapping: dict, mapping from the integer codes to the original song ids
    """
    df = pd.read_csv(path_to_train_users_data)
    # Convert user_id and song_id to categories
    # this the pandas way of mapping strings as integers    
    df["user_id"] = df["user_id"].astype("category")
    df["song_id"] = df["song_id"].astype("category")

    # Extract the integer codes from the categories
    user_codes = df["user_id"].cat.codes
    song_codes = df["song_id"].cat.codes

    # mapping from the integer codes to the original strings
    user_id_mapping = dict(enumerate(df["user_id"].cat.categories))
    song_id_mapping = dict(enumerate(df["song_id"].cat.categories))

    
    if transaction_matrix:
        df["play_count"] = 1  
        # create of type bool to save memory      
        sparse_matrix = csr_matrix(
            (df["play_count"], (user_codes, song_codes)),
            shape=(df["user_id"].cat.categories.size, df["song_id"].cat.categories.size),
            dtype="bool"
        )
    else:
        sparse_matrix = csr_matrix(
            (df["play_count"], (user_codes, song_codes)),
            shape=(df["user_id"].cat.categories.size, df["song_id"].cat.categories.size),
            dtype="int32"
        )
        
    print(f"Created sparse matrix wih shape: {sparse_matrix.shape}")
    print(f"Memory usage of sparse matrix: {sparse_matrix.data.nbytes / 1024:.2f} KB")
    return sparse_matrix , user_id_mapping, song_id_mapping


In [5]:
train_users_data = find_train_users(PATH_ALL_USERS_DATA, PATH_TRAIN_USERS_IDS, PATH_TRAIN_USERS_DATA)
transaction_matrix, user_id_mapping, song_id_mapping = create_sparse_matrix(PATH_TRAIN_USERS_DATA, transaction_matrix=True)
TOTAL_NUMBER_OF_USERS = transaction_matrix.shape[0]


Number of unique users in the training set: 156236
Number of unique songs in the training set: 62066
Created sparse matrix wih shape: (156236, 62066)
Memory usage of sparse matrix: 6432.13 KB


## 6.2 Apriori Algorithm

Our implementation of the Apriori Algorithm to find frequent itemsets of size 2 is as follows:

1. Find the frequent singletons by summing the columns of the transaction matrix and remove the columns of the items which have a support less than the minimum support threshold. Save the frequent singletons.

2. Let A be the transaction matrix only containing the frequent singletons. Now take the matrix product of A and its transpose. The result of this matrix multiplication is a matrix where the element at position (i, j) is the number of transactions where both item i and item j are present. To select the frequent itemsets of size 2, we can extract the row and column indices of the elements that are greater than or equal to the minimum support threshold. Save the frequent itemsets of size 2.

In [6]:
def apriori_algorithm_pairs(crs_matrix: csr_matrix, min_support: float, write_to_file: bool = True):
                            
    """
    Apriori algorithm for pairs using crs_matrix and matrix multiplication
    Input: 
    crs_matrix: sparse matrix, the transaction matrix
    min_support: float, the minimum support
    write_to_file: bool, if True the frequent itemsets are written to a file
    """
    TOTAL_NUMBER_OF_USERS = crs_matrix.shape[0]
    SUPPORT_THRESHOLD = int(math.ceil(TOTAL_NUMBER_OF_USERS * min_support))
    WRITE_TO_FILE = write_to_file
    out_folder = "results"
    os.makedirs(out_folder, exist_ok=True)
    OUT_PATH = f"{out_folder}/frequent_itemsets_support_{min_support}.csv"
    # delete the file if it already exists
    if WRITE_TO_FILE and os.path.exists(OUT_PATH):
        os.remove(OUT_PATH)  
    
    # singletons           
    single_item_occurance = np.array(crs_matrix.sum(axis=0)).flatten()
    relevant_single_item_indices = np.where(single_item_occurance >= SUPPORT_THRESHOLD)[0]
    print(f"Number of single items above the threshold: {len(relevant_single_item_indices)}")
    filtered_csr_idx_to_org_item_idx = {new_index: old_index for new_index, old_index in enumerate(relevant_single_item_indices)}
    filtered_crs_matrix = crs_matrix[:,relevant_single_item_indices]
    # remove old matrix to free up memory
    frequent_singletons = {i: single_item_occurance[i] for i in relevant_single_item_indices}

    if WRITE_TO_FILE:
        with open(OUT_PATH, "a") as f:                
            for singleton, frequency in frequent_singletons.items():
                support = frequency / TOTAL_NUMBER_OF_USERS                             
                f.write(f"({singleton}),{support}\n")

    # pairs
    pairwise_occurance_matrix = filtered_crs_matrix.astype(np.int32).T @ filtered_crs_matrix.astype(np.int32)
    # only extract the lower triangle (excluding the diagonal) since the matrix is symmetric
    pairwise_occurance_matrix = tril(pairwise_occurance_matrix, k=-1)
    coo = pairwise_occurance_matrix.tocoo()
    mask = coo.data >= SUPPORT_THRESHOLD
    filtered_rows, filtered_cols, filtered_values = coo.row[mask], coo.col[mask], coo.data[mask]

    original_rows = [filtered_csr_idx_to_org_item_idx[i] for i in filtered_rows]
    original_cols = [filtered_csr_idx_to_org_item_idx[i] for i in filtered_cols]    

    frequent_pairs  = {}
    for row, col, value in zip(original_rows, original_cols, filtered_values):
        item_set = tuple(sorted((row, col)))
        assert item_set not in frequent_pairs, f"Key {item_set} is already in the dictionary, should not happen"            
        frequent_pairs[item_set] = value

    print(f"Number of pairs above the threshold: {len(frequent_pairs)}")
    if WRITE_TO_FILE:
        with open(OUT_PATH, "a") as f:                
            for itemset in frequent_pairs:
                # Ensure valid mapping of filtered indices to original item indices
                #original_item_indices = tuple(sorted([filtered_csr_idx_to_org_item_idx[i] for i in itemset]))                   
                support = frequent_pairs[itemset] / TOTAL_NUMBER_OF_USERS
                f.write(f"{itemset},{support}\n")        

    return frequent_singletons, frequent_pairs

In [7]:
MIN_SUPPORT = 0.001 # have been run with 0.001, 0,0005, 0.0001, 0.00005
freq_singletons, freq_pairs = apriori_algorithm_pairs(transaction_matrix, min_support=MIN_SUPPORT, write_to_file=True)

support_singletons = {item: freq_singletons[item] / TOTAL_NUMBER_OF_USERS for item in freq_singletons}
support_pairs = {item: freq_pairs[item] / TOTAL_NUMBER_OF_USERS for item in freq_pairs}

singleton_df = pd.DataFrame({"item": list(support_singletons.keys()), "support": list(support_singletons.values())})
pairs_df = pd.DataFrame({"itemset": list(support_pairs.keys()), "pair_support": list(support_pairs.values())})

Number of single items above the threshold: 9228
Number of pairs above the threshold: 47748


## 6.3 Association Rules

In [8]:
def calculate_confidence(support_A_and_B, support_A):
    return support_A_and_B / support_A

def calculate_lift(confidence_A_to_B, support_B):
    return confidence_A_to_B / support_B

pairs_df["item_A"] = pairs_df["itemset"].apply(lambda x: x[0])
pairs_df["item_B"] = pairs_df["itemset"].apply(lambda x: x[1])
pairs_df["support_A"] = pairs_df["item_A"].apply(lambda x: support_singletons[x])
pairs_df["support_B"] = pairs_df["item_B"].apply(lambda x: support_singletons[x])

In [9]:
pairs_df["confidence_A_to_B"] = pairs_df.apply(
    lambda x: calculate_confidence(x["pair_support"], x["support_A"]), axis=1
)
pairs_df["confidence_B_to_A"] = pairs_df.apply(
    lambda x: calculate_confidence(x["pair_support"], x["support_B"]), axis=1)

pairs_df["lift"] = pairs_df.apply(
    lambda x: calculate_lift(x["confidence_A_to_B"], x["support_B"]), axis=1
)
pairs_df.head()

,itemset,pair_support,item_A,item_B,support_A,support_B,confidence_A_to_B,confidence_B_to_A,lift
0,"(14, 55080)",0.001357,14,55080,0.002368,0.008634,0.572973,0.157153,66.359530
1,"(15, 42101)",0.001216,15,42101,0.003168,0.003924,0.383838,0.309951,97.829321
2,"(15, 58716)",0.001146,15,58716,0.003168,0.002944,0.361616,0.389130,122.820571
3,"(18, 39893)",0.001255,18,39893,0.008570,0.004474,0.146378,0.280401,32.717449
4,"(18, 53681)",0.001671,18,53681,0.008570,0.002880,0.194922,0.580000,67.675041


In [10]:
# save to json file:
measurement_dict = {}

for _, row in pairs_df.iterrows():
    item1, item2 = song_id_mapping[row['item_A']], song_id_mapping[row['item_B']]
    conf1_to_2 = row['confidence_A_to_B']
    conf2_to_1 = row['confidence_B_to_A']
    lift = row['lift'] # remember lift_A_to_B == lift_B_to_A    
    # Add item1 -> item2 confidence
    if item1 not in measurement_dict:
        measurement_dict[item1] = {item2: {"confidence": conf1_to_2, "lift": lift}}   
    else:
        measurement_dict[item1][item2] = {"confidence": conf1_to_2, "lift": lift}
    
    # Add item2 -> item1 confidence
    if item2 not in measurement_dict:
        measurement_dict[item2] = {item1: {"confidence": conf2_to_1, "lift": lift}}   
    else:
        measurement_dict[item2][item1] = {"confidence": conf2_to_1, "lift": lift}

# Save to JSON file
with open(f"results/measurements_{MIN_SUPPORT}.json", "w") as json_file:
    json.dump(measurement_dict, json_file, indent=4)

print(f"Saved measurements to results/measurements_{MIN_SUPPORT}.json")

Saved measurements to results/measurements_0.001.json


In [11]:
# restructure the data such that the confidence and lift values are sorted in descending order

def restructure_data(data):
    result = {}
    
    for song, related_songs in data.items():
        # Collect confidence and lift values for each song
        confidence_list = sorted(
            [(related, values['confidence']) for related, values in related_songs.items()],
            key=lambda x: x[1], reverse=True
        )
        lift_list = sorted(
            [(related, values['lift']) for related, values in related_songs.items()],
            key=lambda x: x[1], reverse=True
        )
        
        result[song] = {
            'confidence': confidence_list,
            'lift': lift_list
        }
    
    return result

restructured_data = restructure_data(measurement_dict)
# save to json file:
with open(f"results/measurements_restructured_{MIN_SUPPORT}.json", "w") as json_file:
    json.dump(restructured_data, json_file, indent=4)

## 6.4 Recommendations

To recommend songs to users based on the songs they have listened to, we use the association rules generated by the Apriori Algorithm. We use either confidence or lift metrics to generate the recommendations. For a given test user we have a list of songs that the user has listened to. Based on the users listened songs, we generate 50 song recommendations as follows:

1. For each song the user has listend to, we extract the songs that are associated with the listened song based on either confidence or lift values.

2. Remove the songs that the user has already listened to. Sort the remaining songs based on the confidence or lift values and select the top 50 songs as recommendations.

Finally the recommendations are evaluated using already defined evaluation metrics.


In [12]:
PATH_TEST_DATA_FIXED = "data/processed/listened_songs_fixed_split.csv"
PATH_TEST_MASKED_FIXED = "data/processed/masked_songs_fixed_split.csv"


test_users_data_fixed = pd.read_csv(PATH_TEST_DATA_FIXED)
test_users_data_masked_fixed = pd.read_csv(PATH_TEST_MASKED_FIXED)


def group_data(data):
    """
    Group the data by user_id
    Input:
    data: pd.DataFrame, the data to group
    Returns:
    grouped_data: dict, the grouped data
    """
    grouped_data = data.groupby("user_id")["song_id"].apply(list).reset_index()
    grouped_data.columns = ['user_id', 'songs']
    return grouped_data

test_users_data_fixed = group_data(test_users_data_fixed)
test_users_data_masked_fixed = group_data(test_users_data_masked_fixed)
test_users_data_fixed = test_users_data_fixed.merge(test_users_data_masked_fixed, on="user_id", suffixes=('_listened', '_masked'))
test_users_data_fixed["num_songs_to_recommend"] = test_users_data_fixed["songs_masked"].apply(len)

In [13]:
def recommend_songs(restructured_data,known_songs,k,measurement_type='confidence',verbose=False):
    """
    Recommend k songs based on the known songs
    Input:
    restructured_data: dict, the restructured data that is sorted in descending order
    known_songs: list, the songs the user has listened to
    k: int, the number of songs to recommend
    measurement_type: str, the type of measurement to use, either 'confidence' or 'lift'
    Returns:
    recommended_songs: list, the recommended songs
    """
    # Collect the confidence values for the known songs    
    all_possible_recommendations = []
    for song in known_songs:
        try:
            related_songs = restructured_data[song][measurement_type]
            # remove the songs that are already known
            related_songs = [(related, confidence) for related, confidence in related_songs if related not in known_songs]
            all_possible_recommendations.extend(related_songs)
        except KeyError:
            if verbose:
                print(f"Song {song} is not in the dataset")
            continue
    # Sort the confidence values in descending order
    all_possible_recommendations = sorted(all_possible_recommendations, key=lambda x: x[1], reverse=True)
    # Collect the top k songs
    recommended_songs = [song_and_score for song_and_score in all_possible_recommendations[:k]]
    return recommended_songs
    
    

In [14]:
def add_and_remove_cols(df_in):
    #Make a copy of the dataframe
    df = df_in.copy()
    df["recommended_songs"] = df["recommended_songs_tuple"].apply(lambda x: [song for song, _ in x])
    df["scores"] = df["recommended_songs_tuple"].apply(lambda x: [confidence for _, confidence in x])
    df["probabilities"] = df["scores"].apply(lambda x: np.exp(x) / np.sum(np.exp(x)))
    # keep columns ['user_id', 'recommended_songs', 'recommended_songs_score', 'softmax_score']
    df = df[['user_id', 'recommended_songs', 'scores', 'probabilities']]
    return df

In [15]:
# recommend songs for different measurement types and recommendation strategies
# also evaluate the recommendations
NUM_SONGS_TO_RECOMMEND = 50

test_labels_masked_fixed = pd.read_csv(PATH_TEST_MASKED_FIXED)

for measurement_type in ['confidence', 'lift']:    
    print(f"Measurement type: {measurement_type}")    
    test_users_data_fixed["recommended_songs_tuple"] = test_users_data_fixed.apply(
        lambda row: recommend_songs(restructured_data, 
                                    row["songs_listened"],
                                    NUM_SONGS_TO_RECOMMEND,                                       
                                    measurement_type=measurement_type), axis=1)                                    
    output_fixed = add_and_remove_cols(test_users_data_fixed)
    output_fixed.to_csv(f"results/recommendations_fixed_{measurement_type}_{MIN_SUPPORT}.csv", index=False)
    evaluation_fixed = evaluate_recommendations(output_fixed, 
                                                test_labels_masked_fixed) 
                                                
    evaluation_fixed.to_csv(f"results/evaluation_fixed_{measurement_type}_{MIN_SUPPORT}.csv", index=False)   

Measurement type: confidence
Measurement type: lift


## 6.5 Results


In [16]:
supports = [0.001, 0.0005, 0.0001]
methods = ['confidence', 'lift']

data = []
for support in supports:
    for method in methods:
        file = f"results/evaluation_fixed_{method}_{support}.csv"
        mean_values = pd.read_csv(file).drop(columns=["user_id"]).mean()
        mean_values["method"] = method
        mean_values["support"] = support
        data.append(mean_values)
        
#final creation
final_df = pd.DataFrame(data).reset_index(drop=True)
# sort by support and method
final_df = final_df.sort_values(by=["support", "method"]).reset_index(drop=True)
# make method and support the first columns
final_df = final_df[["method", "support"] + [col for col in final_df.columns if col not in ["method", "support"]]]
final_df

,method,support,k_10,k_20,k_30,k_40,k_50
0,confidence,0.0001,0.156953,0.224492,0.268784,0.301309,0.326465
1,lift,0.0001,0.067148,0.104904,0.134564,0.158955,0.179813
2,confidence,0.0005,0.141647,0.198633,0.235547,0.262005,0.282497
3,lift,0.0005,0.080752,0.122320,0.153533,0.179243,0.200218
4,confidence,0.0010,0.124953,0.172523,0.202466,0.223498,0.240033
5,lift,0.0010,0.081387,0.122244,0.151864,0.174304,0.191843
